In [1]:
import matplotlib.pyplot as plt
import numpy as np
from lo.sdk.api.acquisition.data.decode import SpectralDecoder
from res_enhance import run_resolution_enhancement
import glob
import os
from collections import defaultdict
import spectral

In [2]:
def get_rgb_wavelength_indices(wavelengths):
            """Returns RGB wavelength indices, corresponding to cone response
            from wavelengths in nm.

            "A wide range of colors can be obtained by mixing different amounts of red, green
            and blue light (additive color mixing). A possible combination of wavelengths is
            630 nm for red, 532 nm for green, and 465 nm for blue light."
            Source: https://www.rp-photonics.com/rgb_sources.html

            Args:
                wavelengths (np.ndarray): wavelengths array in nm shape (C).

            Returns:
                RGB Band (np.ndarray): List of RGB bands indices (in that order)
                closest to wavelengths specified.
            """
            return np.asarray([np.argmin(np.abs(wavelengths - i)) for i in [630, 532, 465]])

In [3]:
directory = '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/'

factory_cali = os.path.expanduser('~/Desktop/Rachael_DRAGNet_Project/Metadata/LO1_calibration_files/8_5mm')

date_file = glob.glob(directory + '*')

data_folders = defaultdict(list)


for i in date_file:
    label = i.split('/')[-1]
    blocks = glob.glob(i + '/*BLOCK*')
    
    for j in blocks:
        treatments = glob.glob(j + '/*')
        data_folders[label].append(treatments)

#pop from data folder so it doesnt overload the output
print(data_folders)
#pop here
print(data_folders)

defaultdict(<class 'list'>, {'2024_06_25': [['/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_25/BLOCK_A/DIST', '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_25/BLOCK_A/NPK_DIST', '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_25/BLOCK_A/CONTROL']], '2024_06_26': [['/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_26/BLOCK_A/NPK', '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_26/BLOCK_A/NPK_CESS'], ['/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_26/BLOCK_B/NPK_DIST', '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_26/BLOCK_B/DIST'], ['/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_26/BLOCK_D/NPK_DIST', '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_26/BLOCK_D/DIST']], '2024_06_27': [['/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/2024_06_27/BLOCK_C/NPK_DIST'], ['/Volumes/LaCie/Livin

In [6]:
for date in data_folders:
    blocks = data_folders[date]
    for block in blocks:
        for treatment in block:
            calibration = glob.glob(treatment + '/Calibration/*.loraw')
            decode = SpectralDecoder.from_calibration(factory_cali, calibration[0])
            images = glob.glob(treatment + '/Data/*.loraw')
            for image in images:
                im, scene_view, middle_view = run_resolution_enhancement(image, decode, 'all', 3)

                rgb_idx = get_rgb_wavelength_indices(decode.calibration.wavelengths)
                values = im[:, :, rgb_idx] / im[:, :, rgb_idx].max()

                rgb_idx = get_rgb_wavelength_indices(decode.calibration.wavelengths)

                fig, ax = plt.subplots(1, 3, figsize=(20, 10))
                ax[0].imshow(scene_view)
                ax[0].set_title("Scene view, first frame")
                ax[1].imshow(middle_view[:, :, rgb_idx] / middle_view[:, :, rgb_idx].max())
                ax[1].set_title("Single shot upsampled, first frame")
                ax[2].imshow(values)
                ax[2].set_title("Resolution enhanced")

                #plt.show()
                fig.savefig(directory + 'Enhanced_images/' + '/'.join(treatment.split('/')[-3:]) + '/' + image.split('/')[-1].split('.')[0] + '-enhanced.png')
                plt.close(fig)

                writing_filepath = directory + 'Enhanced_images/' + treatment.split('/')[-3] + '/' + treatment.split('/')[-2] + '/' + treatment.split('/')[-1] 
                print(writing_filepath)



                base_name = image.split('/')[-1].split('.')[0]

                header_file = os.path.join(writing_filepath, base_name + '.hdr')
                spectral.envi.save_image(header_file, im, interleave='bil', dtype=np.float32, force=True)

                np.save(os.path.join(writing_filepath, base_name + "-enhanced.npy"),im)

Running resolution enhancement in homography mode.
Total number of frames in the file: 1


100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


2024_06_25/BLOCK_A/DIST
/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/Enhanced_images/2024_06_25/BLOCK_A/DIST


KeyboardInterrupt: 

In [ ]:
import  spectral.io.envi as envi

dir = '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/Enhanced_images/2024_06_25/BLOCK_A/NPK_DIST/'

img = envi.open(dir + '20240625-110404-743335-single.hdr', dir + '20240625-110404-743335-single.img')

print(img)

	Data Source:   '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/Enhanced_images/2024_06_25/BLOCK_A/NPK_DIST/20240625-110404-743335-single.img'
	# Rows:            637
	# Samples:         635
	# Bands:             3
	Interleave:        BIP
	Quantization:   8 bits
	Data format:     uint8
